https://ramhiser.com/post/2018-04-16-building-scikit-learn-pipeline-with-pandas-dataframe/ https://www.kaggle.com/evanmiller/pipelines-gridsearch-awesome-ml-pipelines

In [339]:
import pandas as pd
import numpy as np
import sklearn
import warnings
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [340]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator ,TransformerMixin 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


In [341]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline ,FeatureUnion ,make_pipeline ,make_union

In [342]:
from sklearn.linear_model import Lasso,Ridge
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [343]:
data_path=os.path.join("..",'..','..','DATA')
input_file=os.path.join(data_path,"california_housing","housing_kaggle.csv")

In [344]:
df=pd.read_csv(input_file)

In [345]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [346]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [347]:
df.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [348]:
X=df.drop('median_house_value',axis=1)
y=df['median_house_value']

In [349]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2)

In [350]:
num_col=(X.select_dtypes(include=np.number)).columns

In [351]:
list(num_col)

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

In [352]:
class TypeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype = dtype

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=[self.dtype])

In [353]:
num_transform=Pipeline([('type_selctor',TypeSelector(np.number)),('SI_median',SimpleImputer(strategy='median',))])

In [354]:
cat_transform=Pipeline([('obj_type_sel',TypeSelector('object')),('SI_most_frequent',SimpleImputer(strategy='most_frequent'))])

In [355]:
feature_union=FeatureUnion(transformer_list=[("num_features_trans",num_transform),("obj_features_trans",cat_transform)])

In [356]:
#num_col=['ocean_proximity']
col=np.arange(0,8,1)

In [357]:
col

array([0, 1, 2, 3, 4, 5, 6, 7])

In [358]:
imp_column_transform=ColumnTransformer([('enc',OneHotEncoder(),[8]),('scale',StandardScaler(),col)],remainder='passthrough')

In [359]:
nn_tr=ColumnTransformer([('enc',OneHotEncoder(),[8])])

In [360]:
final_pipeline=Pipeline([('featureunion',feature_union),('imp_column_transform',imp_column_transform),('lasso',Lasso(alpha=1.0))])
#final_pipeline=Pipeline([('featureunion',feature_union)])

In [361]:
#tranformed_data=final_pipeline.fit_transform(X_train,y_train)

In [362]:
#ct=final_pipeline.named_steps['imp_column_transform']

In [363]:
#hot=ct.named_transformers_['enc']

In [364]:
#hot.get_feature_names()

In [365]:
#hot.categories_

In [366]:
#from scipy import sparse
#A=sparse.csr_matrix(val)

In [367]:
param_grid={'lasso__alpha':[.1,.2,.5,.7]}

In [368]:
gscv=GridSearchCV(final_pipeline,param_grid,cv=3)

In [369]:
gscv.fit(X_train,y_train)

C:\Users\shsubham\AppData\Local\Continuum\anaconda3\envs\tensorflow_cpu\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4270394766356.285, tolerance: 14521699450.313457
  positive)
C:\Users\shsubham\AppData\Local\Continuum\anaconda3\envs\tensorflow_cpu\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4105335988186.629, tolerance: 14480975762.370079
  positive)
C:\Users\shsubham\AppData\Local\Continuum\anaconda3\envs\tensorflow_cpu\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4188268884666.4023, tolerance: 14655728592.082075
  positive)
C:\Users\shsubham\AppData\Local\Continuum\anaconda3\envs\

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('featureunion',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('num_features_trans',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('type_selctor',
                                                                                         TypeSelector(dtype=<class 'numpy.number'>)),
                                                                                        ('SI_median',
                                                                                         SimpleImputer(add_indicator=False,
                                                                                                       copy=True,
                              